<a href="https://colab.research.google.com/github/Sachithra-oshadha/Fabric-Defect-Detection/blob/main/split_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from PIL import Image
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def split_image(image_path, output_dir, tile_size=640, overlap_percent=0.05):
    """Split an image into 640x640 tiles with 5% overlap."""
    img = Image.open(image_path).convert('RGB')
    width, height = img.size
    overlap_pixels = int(tile_size * overlap_percent)
    stride = tile_size - overlap_pixels

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    split_paths = []
    for y in range(0, height, stride):
        for x in range(0, width, stride):
            # Crop tile
            right = min(x + tile_size, width)
            bottom = min(y + tile_size, height)
            tile = img.crop((x, y, right, bottom))

            # Pad if necessary
            if tile.size != (tile_size, tile_size):
                new_tile = Image.new('RGB', (tile_size, tile_size), (0, 0, 0))
                new_tile.paste(tile, (0, 0))
                tile = new_tile

            # Save tile
            tile_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_tile_{y//stride}_{x//stride}.png"
            tile_path = os.path.join(output_dir, tile_name)
            tile.save(tile_path, 'PNG')
            split_paths.append(tile_path)
            logger.info(f"Saved tile {tile_name}")

    return split_paths

def main():
    # Define input folder in mounted Google Drive
    INPUT_FOLDER = '/content/drive/MyDrive/Research'  # Update this path to your folder
    OUTPUT_BASE_DIR = INPUT_FOLDER  # Save splits in the same folder

    # Supported image extensions
    VALID_EXTENSIONS = ('.jpg', '.jpeg', '.png')

    # Get list of images
    image_paths = [
        os.path.join(INPUT_FOLDER, f) for f in os.listdir(INPUT_FOLDER)
        if f.lower().endswith(VALID_EXTENSIONS)
    ]

    if not image_paths:
        logger.error(f"No images found in {INPUT_FOLDER}")
        return

    # Process each image
    for image_path in image_paths:
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        splits_dir = os.path.join(OUTPUT_BASE_DIR, f"{image_name}_splits")
        split_image(image_path, splits_dir)

if __name__ == '__main__':
    main()